In [9]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from pathlib import Path
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [12]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
)
test_datagen = ImageDataGenerator(rescale=1.0/255)
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

In [13]:
train_dir = os.path.join(project_root, 'Image_processing', 'DATASET', 'TRAIN')
test_dir = os.path.join(project_root, 'Image_processing', 'DATASET', 'TEST')

In [14]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    class_mode='binary',
    shuffle=False
)

Found 22565 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [47]:
model = load_model('model.h5')
feature_extractor = tf.keras.Model(inputs= model.input, outputs= model.get_layer('dense_3').output)
feature_extractor.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 124, 124, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 58, 58, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │    13,779,200 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,832,896 (52.77 MB)

 Trainable params: 13,832,896 (52.77 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
X_train = feature_extractor.predict(train_generator)

  1/706 ━━━━━━━━━━━━━━━━━━━━ 1:47 152ms/step

C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


706/706 ━━━━━━━━━━━━━━━━━━━━ 79s 111ms/step


In [49]:
X_test = feature_extractor.predict(test_generator)

79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step


In [50]:
labels = []
for i in tqdm(range(706)):
    inputs, label_batch = next(train_generator)
    labels.extend(label_batch)
labels = np.array(labels)
labels.shape

100%|██████████| 706/706 [00:51<00:00, 13.80it/s]


(22565,)

In [51]:
y_test = []
for i in tqdm(range(79)):
    inputs, label_batch = next(test_generator)
    y_test.extend(label_batch)
y_test = np.array(y_test)
y_test.shape

100%|██████████| 79/79 [00:05<00:00, 13.97it/s]


(2513,)

In [52]:
svm = SVC(kernel='rbf')
svm.fit(X_train, labels)

SVC()

In [53]:
svm.score(X_train[:2000],labels[:2000])

0.557